In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle

pd.set_option('display.max_colwidth', 100)

In [2]:
dpo_path = '/data/wangc239/proteindpo_data/dpo_splits/'

In [3]:
dpo_train = pd.read_csv(os.path.join(dpo_path, '609k_struct_train_dGML.csv'), index_col='Unnamed: 0')
dpo_valid = pd.read_csv(os.path.join(dpo_path, '22k_struct_valid_ddG_ML.csv'), index_col='Unnamed: 0')
dpo_test = pd.read_csv(os.path.join(dpo_path, '25k_struct_test_ddG_ML.csv'), index_col='Unnamed: 0')

In [4]:
dpo_train

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG
0,0.01719762915616707,3.332126,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb,wt,EA:run2_0325_0005.pdb,False,3.178183
5,-0.07229644734890961,3.242632,QEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1Q,D1Q,EA:run2_0325_0005.pdb,False,3.242632
6,-0.0676614464832106,3.247267,EEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1E,D1E,EA:run2_0325_0005.pdb,False,3.247267
7,-0.04581499779145215,3.269113,NEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1N,D1N,EA:run2_0325_0005.pdb,False,3.269113
8,-0.08010508773722425,3.234823,HEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1H,D1H,EA:run2_0325_0005.pdb,False,3.234823
...,...,...,...,...,...,...,...,...
776293,-,-2.850140,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRIVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40I,I32P:L40I,9AME.pdb,-,-2.850140
776294,-,-3.924992,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRWVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40W,I32P:L40W,9AME.pdb,-,-3.924992
776295,-,-2.365774,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRYVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40Y,I32P:L40Y,9AME.pdb,-,-2.365774
776296,-,-1.961486,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRFVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40F,I32P:L40F,9AME.pdb,-,-1.961486


In [5]:
wt_dg_ml = dpo_train.loc[dpo_train['mut_type']=='wt'][['ddG_ML', 'dG_ML', 'WT_name']] # 410 WTs, 375 unique ones
wt1 = [wname for wname in wt_dg_ml['WT_name'] if wt_dg_ml['WT_name'].value_counts().loc[wname] == 1] # 348
dpo_train['WT_name_replace'] = dpo_train['WT_name'].apply(lambda x: x[:2]+'|'+x[3:] if x[2]==':' else x)
wt2 = dpo_train.loc[(dpo_train['mut_type']=='wt') & (dpo_train['WT_name_replace']==dpo_train['name']), 'WT_name'].unique() # 339
wt_keep_single = list(set(wt1) & set(wt2)) # 337
wt_keep_single += ['GG:run5_0637_0006.pdb', 'v2R14S:R16S_2L3X.pdb']

In [6]:
dpo_train_wtkeep = dpo_train.loc[dpo_train['WT_name'].isin(wt_keep_single)]
dpo_train_wtkeep

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,WT_name_replace
0,0.01719762915616707,3.332126,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb,wt,EA:run2_0325_0005.pdb,False,3.178183,EA|run2_0325_0005.pdb
5,-0.07229644734890961,3.242632,QEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1Q,D1Q,EA:run2_0325_0005.pdb,False,3.242632,EA|run2_0325_0005.pdb
6,-0.0676614464832106,3.247267,EEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1E,D1E,EA:run2_0325_0005.pdb,False,3.247267,EA|run2_0325_0005.pdb
7,-0.04581499779145215,3.269113,NEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1N,D1N,EA:run2_0325_0005.pdb,False,3.269113,EA|run2_0325_0005.pdb
8,-0.08010508773722425,3.234823,HEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1H,D1H,EA:run2_0325_0005.pdb,False,3.234823,EA|run2_0325_0005.pdb
...,...,...,...,...,...,...,...,...,...
776293,-,-2.850140,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRIVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40I,I32P:L40I,9AME.pdb,-,-2.850140,9AME.pdb
776294,-,-3.924992,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRWVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40W,I32P:L40W,9AME.pdb,-,-3.924992,9AME.pdb
776295,-,-2.365774,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRYVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40Y,I32P:L40Y,9AME.pdb,-,-2.365774,9AME.pdb
776296,-,-1.961486,NQASVVANQLIPINTALTLVMMRSEVVTPVGPPAEDIPRFVGMQVNRAVPLGTTLMPDMVKGYA,9AME.pdb_dmutv5_32I:40L_I32P:L40F,I32P:L40F,9AME.pdb,-,-1.961486,9AME.pdb


In [7]:
wt_df = dpo_train_wtkeep.loc[dpo_train_wtkeep['mut_type']=='wt'][['ddG_ML', 'dG_ML', 'WT_name', 'name', 'aa_seq']]

In [8]:
wt_df

,ddG_ML,dG_ML,WT_name,name,aa_seq
0,0.01719762915616707,3.332126,EA:run2_0325_0005.pdb,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER
1038,0.0,3.322303,EA:run3_0321_0005.pdb,EA|run3_0321_0005.pdb,HDVTIHAGDKTIHVHGASEEFLRIIEQAKRDGIHEEELLRLIEELVK
2077,-0.22186409863350276,3.717377,EA:run3_0525_0006.pdb,EA|run3_0525_0006.pdb,SHFELRVGTITLHFDNISEELAEELEKLAKNGVHADEIRRWWEEWLK
3103,-0.026321460049135048,3.926922,EA:run3_1140_0005.pdb,EA|run3_1140_0005.pdb,FHVTIHVGDITFHIHGVSEEEVKKLEELVRRGADWREIKKTIEEIEK
5177,-0.023638062327236753,4.091532,EA:run6_1429_0006.pdb,EA|run6_1429_0006.pdb,STVKVRLGHLEVTLHNVSEEDVRELRELWKNGADEEDLRRWLEEWKK
...,...,...,...,...,...
554551,-,5.218107,1IGD.pdb,1IGD.pdb,VTTYKLVINGKTLKGETTTKAVDAETAEKAFKQYANDNGVDGVWTYDDATKTFTVTE
557045,-,5.544929,6NJF.pdb,6NJF.pdb,MTFKLIINGKTLKGETTTEAVDAATAEKVFKQYANDNGLDGEWTYDDATKTFTITE
558280,-,4.702695,5UB0.pdb,5UB0.pdb,MTFKLIINGKTLKGETTTEAVDAATAEKVLKQYANDNGIDGEWTYDDATKTFTVTE
563667,-,5.159033,XX:run8_1203_0001.pdb,XX|run8_1203_0001.pdb,SKIRITWGDKTWEWDGDDEEVWKRILEWARNLGDEELWRHVKEWVDK


In [9]:
# map wt_df to dpo_train_wtkeep based on WT_name to get wt_aa_seq, wt_ddG_ML, wt_dG_ML
dpo_train_wtkeep = dpo_train_wtkeep.merge(wt_df, left_on='WT_name', right_on='WT_name', suffixes=('', '_wt'))
dpo_train_wtkeep

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,WT_name_replace,ddG_ML_wt,dG_ML_wt,name_wt,aa_seq_wt
0,0.01719762915616707,3.332126,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb,wt,EA:run2_0325_0005.pdb,False,3.178183,EA|run2_0325_0005.pdb,0.01719762915616707,3.332126,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER
1,-0.07229644734890961,3.242632,QEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1Q,D1Q,EA:run2_0325_0005.pdb,False,3.242632,EA|run2_0325_0005.pdb,0.01719762915616707,3.332126,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER
2,-0.0676614464832106,3.247267,EEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1E,D1E,EA:run2_0325_0005.pdb,False,3.247267,EA|run2_0325_0005.pdb,0.01719762915616707,3.332126,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER
3,-0.04581499779145215,3.269113,NEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1N,D1N,EA:run2_0325_0005.pdb,False,3.269113,EA|run2_0325_0005.pdb,0.01719762915616707,3.332126,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER
4,-0.08010508773722425,3.234823,HEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,EA|run2_0325_0005.pdb_D1H,D1H,EA:run2_0325_0005.pdb,False,3.234823,EA|run2_0325_0005.pdb,0.01719762915616707,3.332126,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
479097,-,4.106039,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPM,r14_263_TrROS_Hall.pdb_P67M,P67M,r14_263_TrROS_Hall.pdb,-,4.106039,r14_263_TrROS_Hall.pdb,-,5.163265,r14_263_TrROS_Hall.pdb,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPP
479098,-,4.089317,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPL,r14_263_TrROS_Hall.pdb_P67L,P67L,r14_263_TrROS_Hall.pdb,-,4.089317,r14_263_TrROS_Hall.pdb,-,5.163265,r14_263_TrROS_Hall.pdb,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPP
479099,-,5.154741,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPV,r14_263_TrROS_Hall.pdb_P67V,P67V,r14_263_TrROS_Hall.pdb,-,5.154741,r14_263_TrROS_Hall.pdb,-,5.163265,r14_263_TrROS_Hall.pdb,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPP
479100,-,5.138568,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPI,r14_263_TrROS_Hall.pdb_P67I,P67I,r14_263_TrROS_Hall.pdb,-,5.138568,r14_263_TrROS_Hall.pdb,-,5.163265,r14_263_TrROS_Hall.pdb,MRIRIEVETRTGPDGRKYVVVRIRDEKTGEELELVFESREEAEEAAKALAEELKKAGYEIEVEIRPP


In [10]:
dpo_train_dict_wt = {}
for i, row in tqdm(dpo_train_wtkeep.iterrows()):
    name = row['name']
    if row['mut_type'] == 'wt': 
        dpo_train_dict_wt[name] = [row['aa_seq'], row['WT_name'], row['aa_seq_wt'], row['dG_ML'], row['ddG_ML'], row['dG_ML_wt'], row['ddG_ML_wt'], row['name_wt']]

479102it [00:18, 25899.27it/s]


In [18]:
# save_dir = '/data/wangc239/proteindpo_data/processed_data/'

with open(os.path.join(save_dir, 'dpo_train_dict_wt.pkl'), 'wb') as f:
    pickle.dump(dpo_train_dict_wt, f)

In [25]:
# create a dictionary with keys as name and values as list of [aa_seq, wt_name, aa_seq_wt, dG_ML, ddG_ML, dG_ML_wt, ddG_ML_wt, name_wt]
dpo_train_dict = {}
for i, row in tqdm(dpo_train_wtkeep.iterrows()):
    name = row['name']
    dpo_train_dict[name] = [row['aa_seq'], row['WT_name'], row['aa_seq_wt'], row['dG_ML'], row['ddG_ML'], row['dG_ML_wt'], row['ddG_ML_wt'], row['name_wt']]

479102it [00:26, 18094.59it/s]


In [27]:
# save the dictionary
save_dir = '/data/wangc239/proteindpo_data/processed_data/'
os.makedirs(save_dir, exist_ok=True)
with open(os.path.join(save_dir, 'dpo_train_dict.pkl'), 'wb') as f:
    pickle.dump(dpo_train_dict, f)

### Explore other dropped wts

In [111]:
set(wt1) - set(wt2) # the first 9 are the cases that the wt is actually a mutant

{'1SF0.pdb',
 '1UCS.pdb',
 '1YEZ.pdb',
 '2KZI.pdb',
 '2LX2.pdb',
 '2M7O.pdb',
 '2MLB.pdb',
 '5XR0.pdb',
 '6NS8.pdb',
 'GG:run5_0637_0006.pdb',
 'v2R14S:R16S_2L3X.pdb'}

In [135]:
set(wt2) - set(wt1)

{'2B88.pdb', '2HBB.pdb'}

In [136]:
dpo_train.loc[dpo_train['WT_name']=='2B88.pdb']

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,WT_name_replace
105608,0.0,2.313166,NKFNKELGWATWEIFNLPNLNGVQVKAFIDSLRDDPSQSANLLAEAKKLNDAQA,2B88.pdb,wt,2B88.pdb,False,2.352405,2B88.pdb
105613,-0.13238321680971232,2.180782,QKFNKELGWATWEIFNLPNLNGVQVKAFIDSLRDDPSQSANLLAEAKKLNDAQA,2B88.pdb_N1Q,N1Q,2B88.pdb,False,2.180782,2B88.pdb
105614,0.019448899558130783,2.332615,EKFNKELGWATWEIFNLPNLNGVQVKAFIDSLRDDPSQSANLLAEAKKLNDAQA,2B88.pdb_N1E,N1E,2B88.pdb,False,2.332615,2B88.pdb
105615,0.025446052699495958,2.338612,HKFNKELGWATWEIFNLPNLNGVQVKAFIDSLRDDPSQSANLLAEAKKLNDAQA,2B88.pdb_N1H,N1H,2B88.pdb,False,2.338612,2B88.pdb
105616,0.15503393539464394,2.468200,DKFNKELGWATWEIFNLPNLNGVQVKAFIDSLRDDPSQSANLLAEAKKLNDAQA,2B88.pdb_N1D,N1D,2B88.pdb,False,2.468200,2B88.pdb
...,...,...,...,...,...,...,...,...,...
382234,-0.09055180993855139,3.152200,NKFNKELQWATWEILNLPNLNEEQVKAFIDSLKDDPSQSANLLAEAKKLNDAQI,2B88.pdb_pross4_A54I,A54I,2B88.pdb,False,3.152200,2B88.pdb
382235,-0.030235536202444457,3.212516,NKFNKELQWATWEILNLPNLNEEQVKAFIDSLKDDPSQSANLLAEAKKLNDAQW,2B88.pdb_pross4_A54W,A54W,2B88.pdb,False,3.212516,2B88.pdb
382236,-0.13058371980931716,3.112168,NKFNKELQWATWEILNLPNLNEEQVKAFIDSLKDDPSQSANLLAEAKKLNDAQY,2B88.pdb_pross4_A54Y,A54Y,2B88.pdb,False,3.112168,2B88.pdb
382237,-0.001612225566899106,3.241139,NKFNKELQWATWEILNLPNLNEEQVKAFIDSLKDDPSQSANLLAEAKKLNDAQF,2B88.pdb_pross4_A54F,A54F,2B88.pdb,False,3.241139,2B88.pdb


In [137]:
dpo_train.loc[dpo_train['WT_name']=='2HBB.pdb']

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,WT_name_replace
106800,0.016269242253122762,2.332296,MKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb,wt,2HBB.pdb,False,2.323650,2HBB.pdb
106805,-1.4635537665870963,0.852473,QKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1Q,M1Q,2HBB.pdb,False,0.852473,2HBB.pdb
106806,-3.002679940335729,-0.686654,EKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1E,M1E,2HBB.pdb,False,-0.686654,2HBB.pdb
106807,-2.4541776757511213,-0.138151,NKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1N,M1N,2HBB.pdb,False,-0.138151,2HBB.pdb
106808,-2.456164731344052,-0.140138,HKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1H,M1H,2HBB.pdb,False,-0.140138,2HBB.pdb
...,...,...,...,...,...,...,...,...,...
623569,-,-2.238580,MKVIFLKDVKGKGKKGPIINVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_hnet1_E17P:K19I,E17P:K19I,2HBB.pdb,False,-2.238580,2HBB.pdb
623570,-,-3.820932,MKVIFLKDVKGKGKKGPIWNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_hnet1_E17P:K19W,E17P:K19W,2HBB.pdb,False,-3.820932,2HBB.pdb
623571,-,-1.212211,MKVIFLKDVKGKGKKGPIYNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_hnet1_E17P:K19Y,E17P:K19Y,2HBB.pdb,False,-1.212211,2HBB.pdb
623572,-,-1.747260,MKVIFLKDVKGKGKKGPIFNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_hnet1_E17P:K19F,E17P:K19F,2HBB.pdb,False,-1.747260,2HBB.pdb


In [145]:
dpo_train.loc[dpo_train['WT_name']=='2HBB.pdb'].iloc[:905]

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,WT_name_replace
106800,0.016269242253122762,2.332296,MKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb,wt,2HBB.pdb,False,2.323650,2HBB.pdb
106805,-1.4635537665870963,0.852473,QKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1Q,M1Q,2HBB.pdb,False,0.852473,2HBB.pdb
106806,-3.002679940335729,-0.686654,EKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1E,M1E,2HBB.pdb,False,-0.686654,2HBB.pdb
106807,-2.4541776757511213,-0.138151,NKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1N,M1N,2HBB.pdb,False,-0.138151,2HBB.pdb
106808,-2.456164731344052,-0.140138,HKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALE,2HBB.pdb_M1H,M1H,2HBB.pdb,False,-0.140138,2HBB.pdb
...,...,...,...,...,...,...,...,...,...
107852,0.25836958313827196,2.574396,MKVIFLKDVKGKGKKGEIKNVADGYANNFLFKQGLAIEATPANLKALC,2HBB.pdb_E48C,E48C,2HBB.pdb,-,2.574396,2HBB.pdb
382240,-1.630857303141721,1.779522,QKVIFLKDVKGKGKKGEIKNVADGYARNFLFKQGLAIEATPANLKALE,2HBB.pdb_pross4_M1Q,M1Q,2HBB.pdb,False,1.779522,2HBB.pdb
382241,-2.046937117866163,1.363442,EKVIFLKDVKGKGKKGEIKNVADGYARNFLFKQGLAIEATPANLKALE,2HBB.pdb_pross4_M1E,M1E,2HBB.pdb,False,1.363442,2HBB.pdb
382242,-1.6741696120725749,1.736209,NKVIFLKDVKGKGKKGEIKNVADGYARNFLFKQGLAIEATPANLKALE,2HBB.pdb_pross4_M1N,M1N,2HBB.pdb,False,1.736209,2HBB.pdb


In [149]:
set(wt_dg_ml['WT_name']) - set(wt1) - set(wt2) # 25
# they are the ones that have multiple mutants labelled as wt and mutations on top of it

{'1A0N.pdb',
 '1BK2.pdb',
 '1CSQ.pdb',
 '1EM7.pdb',
 '1GB4.pdb',
 '1H8K.pdb',
 '1MJC.pdb',
 '1OPS.pdb',
 '1PGA.pdb',
 '1PGX.pdb',
 '1S1N.pdb',
 '1SIF.pdb',
 '1UBQ.pdb',
 '2K52.pdb',
 '2K5H.pdb',
 '2LSS.pdb',
 '2LXK.pdb',
 '2LYP.pdb',
 '2MKX.pdb',
 '2PTL.pdb',
 '2ROT.pdb',
 '2ZW1.pdb',
 '5ECA.pdb',
 '5FWB.pdb',
 '5GU9.pdb'}

### valid and test

##### valid EDA

In [202]:
valid_wt_dg_ml = dpo_valid.loc[dpo_valid['mut_type']=='wt'][['ddG_ML', 'dG_ML', 'WT_name']] # 16 WTs, 13 unique ones

In [207]:
dpo_valid.loc[(dpo_valid['mut_type']=='wt') & (dpo_valid['WT_name']==dpo_valid['name']), 'WT_name'].unique() # 9 (actually 10)

array(['1ZHC.pdb', '1UZC.pdb', '2LQK.pdb', '2MCK.pdb', '1JIC.pdb',
       '2WQG.pdb', 'v2_4UZX.pdb', '4UZW.pdb', '2MCH.pdb'], dtype=object)

In [206]:
valid_wt_dg_ml.WT_name.value_counts()

WT_name
2KRS.pdb                2
2KT8.pdb                2
2KYB.pdb                2
1ZHC.pdb                1
1UZC.pdb                1
2LQK.pdb                1
2MCK.pdb                1
1JIC.pdb                1
2WQG.pdb                1
v2R31S:R32S_2N5D.pdb    1
v2_4UZX.pdb             1
4UZW.pdb                1
2MCH.pdb                1
Name: count, dtype: int64

In [210]:
dpo_valid.loc[(dpo_valid['WT_name']=='2KRS.pdb') & (dpo_valid['mut_type']=='wt')]

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG
324711,0.000000,3.073411,MQGVVKVNSALNMRSGPGSNYGVIGTLRNNDKVEIIKEVDGAYEIRFNGKVGYASKSYITIVN,2KRS.pdb_W42A,wt,2KRS.pdb,False,3.056207
326098,-0.071952,2.442280,MQGVVKVNSALNMRSGPGSNYGVIGTLRNNDKVEIIKEVDGWYEIRFNGKVGSASKSYITIVN,2KRS.pdb_Y53S,wt,2KRS.pdb,False,2.496740


In [211]:
dpo_valid.loc[(dpo_valid['WT_name']=='2KT8.pdb') & (dpo_valid['mut_type']=='wt')]

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG
327488,-0.036311,3.494454,AEKTGIVNVSSSLNVREGASTSSKVIGSLSGNTKVTIVGEEGAFYKIEYKGSHSYVAKEYIKDIKDE,2KT8.pdb_G54S,wt,2KT8.pdb,False,3.524885
328967,0.073359,2.184647,AEKTGIVNVSSSLNAREGASTSSKVIGSLSGNTKVTIVGEEGAFYKIEYKGSHGYVAKEYIKDIKDE,2KT8.pdb_V15A,wt,2KT8.pdb,False,2.124580


In [212]:
dpo_valid.loc[(dpo_valid['WT_name']=='2KYB.pdb') & (dpo_valid['mut_type']=='wt')]

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG
330443,-0.006665,3.497715,TSIVNVSSSLNVRSSASTSSKVIGSLSGNTKVTIVGEEGAFYKIEYKGSHGYVAKEYI,2KYB.pdb_G2S,wt,2KYB.pdb,False,3.508628
331723,0.000000,3.155036,TGIVNVSSSLNVRSSASTSSKVIGSLSGNTKVTIVGEEGAFYKIEYKGSHGYVAAEYI,2KYB.pdb_K55A,wt,2KYB.pdb,False,3.184007


#### valid dict

In [11]:
dpo_valid_wtkeep = dpo_valid.loc[dpo_valid['WT_name'].isin(['1ZHC.pdb', '1UZC.pdb', '2LQK.pdb', '2MCK.pdb', '1JIC.pdb',
       '2WQG.pdb', 'v2_4UZX.pdb', '4UZW.pdb', '2MCH.pdb', 'v2R31S:R32S_2N5D.pdb'])]
wt_df_valid = dpo_valid_wtkeep.loc[dpo_valid_wtkeep['mut_type']=='wt'][['ddG_ML', 'dG_ML', 'WT_name', 'name', 'aa_seq']]

In [12]:
dpo_valid_wtkeep = dpo_valid_wtkeep.merge(wt_df_valid, left_on='WT_name', right_on='WT_name', suffixes=('', '_wt'))
dpo_valid_wtkeep

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,ddG_ML_wt,dG_ML_wt,name_wt,aa_seq_wt
0,-0.028266,0.985102,HEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK,1ZHC.pdb,wt,1ZHC.pdb,False,1.062394,-0.028266,0.985102,1ZHC.pdb,HEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK
1,-0.226516,0.786853,QEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK,1ZHC.pdb_H1Q,H1Q,1ZHC.pdb,False,0.786853,-0.028266,0.985102,1ZHC.pdb,HEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK
2,0.219706,1.233074,EEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK,1ZHC.pdb_H1E,H1E,1ZHC.pdb,-,1.233074,-0.028266,0.985102,1ZHC.pdb,HEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK
3,-0.178393,0.834975,NEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK,1ZHC.pdb_H1N,H1N,1ZHC.pdb,False,0.834975,-0.028266,0.985102,1ZHC.pdb,HEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK
4,0.262875,1.276244,DEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK,1ZHC.pdb_H1D,H1D,1ZHC.pdb,-,1.276244,-0.028266,0.985102,1ZHC.pdb,HEFRDEISVLKANNPHFDKIFEKHNQLDDDIKTAEQQNASDAEVSHMKKQKLKLKDEIHSMIIEYREK
...,...,...,...,...,...,...,...,...,...,...,...,...
15177,-2.404054,1.197158,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLPMIDKAIFRRYPHLR,2MCH.pdb_hnet1_D50P:S52I,D50P:S52I,2MCH.pdb,False,1.197158,0.060347,3.661558,2MCH.pdb,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLDMSDKAIFRRYPHLR
15178,-2.772238,0.828973,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLPMWDKAIFRRYPHLR,2MCH.pdb_hnet1_D50P:S52W,D50P:S52W,2MCH.pdb,False,0.828973,0.060347,3.661558,2MCH.pdb,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLDMSDKAIFRRYPHLR
15179,-2.943402,0.657810,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLPMYDKAIFRRYPHLR,2MCH.pdb_hnet1_D50P:S52Y,D50P:S52Y,2MCH.pdb,False,0.657810,0.060347,3.661558,2MCH.pdb,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLDMSDKAIFRRYPHLR
15180,-2.450802,1.150409,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLPMFDKAIFRRYPHLR,2MCH.pdb_hnet1_D50P:S52F,D50P:S52F,2MCH.pdb,False,1.150409,0.060347,3.661558,2MCH.pdb,HHHHHGSGALAALHAEGPLAGLPVTRSDARVLIFNEWEERKKSDPWLRLDMSDKAIFRRYPHLR


In [13]:
dpo_valid_dict_wt = {}
for i, row in tqdm(dpo_valid_wtkeep.iterrows()):
    name = row['name']
    if row['mut_type'] == 'wt': 
        dpo_valid_dict_wt[name] = [row['aa_seq'], row['WT_name'], row['aa_seq_wt'], row['dG_ML'], row['ddG_ML'], row['dG_ML_wt'], row['ddG_ML_wt'], row['name_wt']]

15182it [00:00, 20315.22it/s]


In [19]:
with open(os.path.join(save_dir, 'dpo_valid_dict_wt.pkl'), 'wb') as f:
    pickle.dump(dpo_valid_dict_wt, f)

In [17]:
dpo_valid_dict = {}
for i, row in tqdm(dpo_valid_wtkeep.iterrows()):
    name = row['name']
    dpo_valid_dict[name] = [row['aa_seq'], row['WT_name'], row['aa_seq_wt'], row['dG_ML'], row['ddG_ML'], row['dG_ML_wt'], row['ddG_ML_wt'], row['name_wt']]

15182it [00:00, 19611.35it/s]


In [33]:
with open(os.path.join(save_dir, 'dpo_valid_dict.pkl'), 'wb') as f:
    pickle.dump(dpo_valid_dict, f)

#### test EDA

In [214]:
test_wt_dg_ml = dpo_test.loc[dpo_test['mut_type']=='wt'][['ddG_ML', 'dG_ML', 'WT_name']] # 14 WTs, 14 unique ones

In [216]:
test_wt_dg_ml.WT_name.value_counts()

WT_name
EA:run5_0050_0004.pdb    1
XX:run7_0974_0003.pdb    1
r6_560_TrROS_Hall.pdb    1
HEEH_KT_rd6_0746.pdb     1
2M2J.pdb                 1
2MA4.pdb                 1
1F0M.pdb                 1
5JRT.pdb                 1
7JJK.pdb                 1
2KRU.pdb                 1
2L09.pdb                 1
4G3O.pdb                 1
2LVN.pdb                 1
v2K43S_2KVV.pdb          1
Name: count, dtype: int64

In [217]:
dpo_test.loc[(dpo_test['mut_type']=='wt') & (dpo_test['WT_name']==dpo_test['name']), 'WT_name'].unique() # 12 (actually 14)

array(['r6_560_TrROS_Hall.pdb', 'HEEH_KT_rd6_0746.pdb', '2M2J.pdb',
       '2MA4.pdb', '1F0M.pdb', '5JRT.pdb', '7JJK.pdb', '2KRU.pdb',
       '2L09.pdb', '4G3O.pdb', '2LVN.pdb', 'v2K43S_2KVV.pdb'],
      dtype=object)

#### test dict

In [14]:
dpo_test_wtkeep = dpo_test.copy()
wt_df_test = dpo_test_wtkeep.loc[dpo_test_wtkeep['mut_type']=='wt'][['ddG_ML', 'dG_ML', 'WT_name', 'name', 'aa_seq']]

In [15]:
dpo_test_wtkeep = dpo_test_wtkeep.merge(wt_df_test, left_on='WT_name', right_on='WT_name', suffixes=('', '_wt'))
dpo_test_wtkeep

,ddG_ML,dG_ML,aa_seq,name,mut_type,WT_name,Stabilizing_mut,deltaG,ddG_ML_wt,dG_ML_wt,name_wt,aa_seq_wt
0,0.192780,4.381241,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK,EA|run5_0050_0004.pdb,wt,EA:run5_0050_0004.pdb,False,4.221573,0.19278,4.381241,EA|run5_0050_0004.pdb,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK
1,0.046238,4.234700,QEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK,EA|run5_0050_0004.pdb_T1Q,T1Q,EA:run5_0050_0004.pdb,False,4.234700,0.19278,4.381241,EA|run5_0050_0004.pdb,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK
2,0.052832,4.241293,EEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK,EA|run5_0050_0004.pdb_T1E,T1E,EA:run5_0050_0004.pdb,False,4.241293,0.19278,4.381241,EA|run5_0050_0004.pdb,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK
3,0.161277,4.349739,NEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK,EA|run5_0050_0004.pdb_T1N,T1N,EA:run5_0050_0004.pdb,False,4.349739,0.19278,4.381241,EA|run5_0050_0004.pdb,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK
4,0.056611,4.245073,HEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK,EA|run5_0050_0004.pdb_T1H,T1H,EA:run5_0050_0004.pdb,False,4.245073,0.19278,4.381241,EA|run5_0050_0004.pdb,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK
...,...,...,...,...,...,...,...,...,...,...,...,...
23463,-0.342189,3.186652,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIW,v2K43S_2KVV.pdb_R61W,R61W,v2K43S_2KVV.pdb,False,3.186652,0.00000,3.528841,v2K43S_2KVV.pdb,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIR
23464,-0.337174,3.191667,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIY,v2K43S_2KVV.pdb_R61Y,R61Y,v2K43S_2KVV.pdb,False,3.191667,0.00000,3.528841,v2K43S_2KVV.pdb,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIR
23465,-0.288015,3.240826,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIF,v2K43S_2KVV.pdb_R61F,R61F,v2K43S_2KVV.pdb,False,3.240826,0.00000,3.528841,v2K43S_2KVV.pdb,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIR
23466,-1.908796,1.620045,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIP,v2K43S_2KVV.pdb_R61P,R61P,v2K43S_2KVV.pdb,False,1.620045,0.00000,3.528841,v2K43S_2KVV.pdb,EEWMVEKALMVRTGLGARQIESYRQGAWIEGVHFKRVSPSGESTLRGTTWYNYPEINKFIR


In [16]:
dpo_test_dict_wt = {}
for i, row in tqdm(dpo_test_wtkeep.iterrows()):
    name = row['name']
    if row['mut_type'] == 'wt': 
        dpo_test_dict_wt[name] = [row['aa_seq'], row['WT_name'], row['aa_seq_wt'], row['dG_ML'], row['ddG_ML'], row['dG_ML_wt'], row['ddG_ML_wt'], row['name_wt']]

23468it [00:00, 25993.93it/s]


In [20]:
with open(os.path.join(save_dir, 'dpo_test_dict_wt.pkl'), 'wb') as f:
    pickle.dump(dpo_test_dict_wt, f)

In [38]:
dpo_test_dict = {}
for i, row in tqdm(dpo_test_wtkeep.iterrows()):
    name = row['name']
    dpo_test_dict[name] = [row['aa_seq'], row['WT_name'], row['aa_seq_wt'], row['dG_ML'], row['ddG_ML'], row['dG_ML_wt'], row['ddG_ML_wt'], row['name_wt']]

23468it [00:01, 18891.93it/s]


In [39]:
with open(os.path.join(save_dir, 'dpo_test_dict.pkl'), 'wb') as f:
    pickle.dump(dpo_test_dict, f)

In [51]:
wt_df_test['aa_seq'].apply(lambda x: len(x)).max()

71